In [6]:
import pypsa
import matplotlib.pyplot as plt
import pandas as pd
import myUtils as utils
tech = "biogas"  #biogas,battery, biomass, ror, geothermal，transmission
rootPath = f"../../results/scenario/{tech}"
pathTail = "/networks/base_s_27_3H_3H_2045.nc"


plt.style.use('bmh')
%matplotlib inline

nBase = pypsa.Network(f"../../results/scenario/base{pathTail}")
nMean = pypsa.Network(f"{rootPath}_mean/{pathTail}")
nGood = pypsa.Network(f"{rootPath}_good/{pathTail}")
nBest = pypsa.Network(f"{rootPath}_best/{pathTail}")

networks = {
  "nBase": nBase,
  "nMean": nMean,
  "nGood": nGood,
  "nBest": nBest
}

INFO:pypsa.io:Imported network base_s_27_3H_3H_2045.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores
INFO:pypsa.io:Imported network base_s_27_3H_3H_2045.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores
INFO:pypsa.io:Imported network base_s_27_3H_3H_2045.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores
INFO:pypsa.io:Imported network base_s_27_3H_3H_2045.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores


In [7]:
import pandas as pd
import myUtils as utils
import importlib
importlib.reload(utils)


def getDfValue (n):
  lists = [
    n.statistics.capex().sum() / 1e9,
    n.statistics.capex(groupby="de_grouper").sum() / 1e9,
    n.statistics.expanded_capex(groupby="de_grouper").sum() / 1e9,
    n.statistics.opex(groupby="de_grouper").sum() / 1e9,

    n.statistics.capex(groupby="de_all_generator_grouper").sum() / 1e9,
    n.statistics.opex(groupby="de_all_generator_grouper").sum() / 1e9,
  ]
  return lists

data = {
  'Name': [
    "CAPEX(inkl. Nachbarländer)",
    "CAPEX",
    "investierte CAPEX",
    "OPEX",

    "Strom CAPEX",
    "Strom Opex"
  ],
  # 'Unit': [
  #   'Mrd.EUR',
  #   'Mrd.EUR',
  #   'Mrd.EUR',
  #   'Mrd.EUR',
  #   'Mrd.EUR',
  #   'Mrd.EUR'
  # ],
  'Base': getDfValue(nBase),
  'Mean': getDfValue(nMean),
  'Optimist': getDfValue(nGood),
  'Best': getDfValue(nBest)
}
print(len(data['Name']), len(data['Name']), len(data['Base']))



df = pd.DataFrame(data).set_index('Name')

caepx_base = df.loc['CAPEX', 'Base']
caepx_changes = (df.loc['CAPEX', ['Mean', 'Optimist', 'Best']] - caepx_base) / caepx_base * 100

elc_caepx_base = df.loc['Strom CAPEX', 'Base']
elc_caepx_changes = (df.loc['Strom CAPEX', ['Mean', 'Optimist', 'Best']] - elc_caepx_base) / elc_caepx_base * 100

table = utils.dataframe_to_table(df)
print(table)
print(caepx_changes)
print(elc_caepx_changes)

6 6 6
  [CAPEX(inkl. Nachbarländer)], [637,328], [606,441], [606,756], [603,359],
  [CAPEX], [74,925], [69,730], [69,670], [69,430],
  [investierte CAPEX], [63,338], [58,143], [58,084], [57,843],
  [OPEX], [5,494], [7,397], [7,033], [6,417],
  [Strom CAPEX], [35,375], [32,166], [32,166], [32,166],
  [Strom Opex], [0,131], [0,136], [0,136], [0,136],
Mean       -6.93368
Optimist   -7.01305
Best       -7.33459
Name: CAPEX, dtype: float64
Mean       -9.07135
Optimist   -9.07135
Best       -9.07139
Name: Strom CAPEX, dtype: float64


In [8]:
import pandas as pd
import myUtils as utils
import importlib
importlib.reload(utils)

df = pd.DataFrame()

for key, n in networks.items():
  generator_df = utils.getNByGroup(n, 'Generator', utils.de_generator_grouper)
  generators = generator_df.groupby('carrier').sum().p_nom_opt / 1e3
  g_links_df = utils.getNByGroup(n, 'Link', utils.de_generator_grouper)
  generatorLinks = g_links_df.groupby('carrier').sum().p_nom_opt / 1e3

  h2_df = utils.getNByGroup(n, 'Link', utils.de_h2_dispatch_grouper)
  h2s =  h2_df.groupby('carrier').sum().p_nom_opt / 1e3

  biomass = utils.getNByGroup(n,'Link',utils.de_biomass_chp)
  # print(biomass)
  s = pd.concat([generators, generatorLinks,h2s])
  s['Offwind'] = s['offwind-ac'] + s['offwind-dc'] + s['offwind-float']
  s['Solar'] = s['solar'] + s['solar-hsat']
  s['Onwind'] = s['onwind']
  

  s['Solardach'] = s['solar rooftop']
  s['Erdgas CHP'] = s['urban central CHP']
  s['Geothermal'] = s.get('geothermal organic rankine cycle', 0)
  s['Biomasse CHP'] = biomass.p_nom_opt.sum() 
  s['Gesamt'] = s.sum()
  
  s = s.reindex(['Onwind', 'Offwind', 'Solar', 'Solardach','OCGT','Erdgas CHP', 'Geothermal','H2 Fuel Cell', 'H2 turbine', 'Biomasse CHP', 'ror', 'Gesamt'])
  df[key] = s

# df = df[(df != 0).any(axis=1)]
df = df.fillna(0)


print(utils.dataframe_to_table(df))

print(df)


  [Onwind], [83,101], [63,070], [63,070], [63,070],
  [Offwind], [78,329], [78,329], [78,329], [78,329],
  [Solar], [50,287], [50,287], [50,287], [50,287],
  [Solardach], [107,327], [90,006], [90,006], [90,006],
  [OCGT], [73,804], [69,800], [69,800], [69,800],
  [Erdgas CHP], [61,302], [50,328], [50,328], [50,328],
  [Geothermal], [0,000], [0,000], [0,000], [0,000],
  [H2 Fuel Cell], [0,000], [0,000], [0,000], [0,000],
  [H2 turbine], [0,000], [0,000], [0,000], [0,000],
  [Biomasse CHP], [0,000], [0,000], [0,000], [0,000],
  [ror], [0,000], [0,000], [0,000], [0,000],
  [Gesamt], [834,497], [733,842], [733,842], [733,841],
                 nBase     nMean     nGood     nBest
carrier                                             
Onwind        83.10055  63.06995  63.06993  63.06995
Offwind       78.32878  78.32878  78.32878  78.32878
Solar         50.28742  50.28734  50.28736  50.28733
Solardach    107.32740  90.00627  90.00644  90.00578
OCGT          73.80419  69.80010  69.79988  69.8005